<a href="https://colab.research.google.com/github/yhatpub/yhatpub/blob/errors/notebooks/fastai/incorrect_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

incorrect model download

In [ ]:
!pip install -q --upgrade --no-cache-dir fastai
!pip install -q spacy==2.2.4
!pip install --no-cache-dir git+https://github.com/yhatpub/yhat_params.git@main

In [ ]:
from fastai.vision.all import *
from yhat_params.yhat_tools import FieldType, inference_predict

this is the model from fastai lesson10

In [ ]:
#cleanup from previous download
!rm -f uc*

#file copied from google drive
google_drive_url = "https://drive.google.com/file/d/1rs1lYSiVBOG_ZMso2mvIMZaWCQxcDikx/view?usp=sharing"

import os
os.environ['GOOGLE_FILE_ID'] = google_drive_url.split('/')[5]
os.environ['GDRIVE_URL'] = f'https://docs.google.com/uc?export=download&id={os.environ["GOOGLE_FILE_ID"]}'
!echo "This is the Google drive download url $GDRIVE_URL"

In [ ]:
!wget --no-check-certificate $GDRIVE_URL -r -A 'uc*' -e robots=off -nd
!mkdir -p model
!mv $(ls -S uc* | head -1) ./model/export.pkl

In [ ]:
!ls -l ./model/export.pkl

In [ ]:
learn_inf = load_learner('./model/export.pkl')

In [ ]:
input = {"image": FieldType.PIL}
output = {"text": FieldType.Text}

@inference_predict(input=input, output=output)
def predict(params):
    img = PILImage.create(np.array(params["image"].convert("RGB")))
    pred,pred_idx,probs = learn_inf.predict(img)
    return {"text":  f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'}

In [ ]:
from yhat_params.yhat_tools import in_colab, inference_test

if in_colab():
    import urllib.request
    from PIL import Image
    print("about to download image")
    urllib.request.urlretrieve("https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/GrizzlyBearJeanBeaufort.jpg/220px-GrizzlyBearJeanBeaufort.jpg", "input_image.jpg")
    img = Image.open("input_image.jpg")
    print("about to run inference_test")
    inference_test(predict_func=predict, params={'image': img})